In [664]:

import pandas as pd
import cv2
import numpy as np



class driveFile:
  def __init__(self):
    self.path=""
  def dataload(self,folder_path):
    img=[]
    label=[]
    import os
    self.path1=os.path.join(self.path,folder_path)
    
    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    
    img_array=np.array(img)
    return img_array

In [ ]:

drive=driveFile()
images,labels=drive.dataload('dataset/train_set')
test_images,test_labels=drive.dataload('dataset/test_set')
images = (images - np.mean(images)) / np.std(images)
test_images = (test_images - np.mean(test_images)) / np.std(test_images)


In [666]:
images=np.array([img.flatten() for img in images])
test_images=np.array([img.flatten() for img in test_images])
labels=np.array(labels)
test_labels=np.array(test_labels)

unique_labels = np.unique(labels)
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

integer_encoded = np.array([label_to_index[label] for label in labels])
test_integer_encoded = np.array([label_to_index[label] for label in test_labels])
one_hot_encoded = np.eye(len(unique_labels))[integer_encoded]
test_one_hot_encoded = np.eye(len(unique_labels))[test_integer_encoded]
labels=one_hot_encoded
test_labels=test_one_hot_encoded


In [667]:
class Layer:
    def __init__(self):
        self.input=None
        self.output=None
    def forward(self,input):
        # to be overridden 
        pass 
    def backward(self,output_gradient,learning_rate):
        pass
    

In [668]:
class Dense(Layer):
    def __init__(self,input_size,output_size):
        # self.weights=np.random.randn(input_size,output_size)*0.01
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / (input_size + output_size))
        self.bias=np.random.randn(1,output_size)
        self.velocities_W=np.zeros_like(self.weights)
        self.velocities_B=np.zeros_like(self.bias)
        
    def forward(self,input):
        self.input=input
        return np.dot(self.input,self.weights)+self.bias
    
    def backward(self,output_gradient,learning_rate,momentcoeff):
        w_gradient = np.dot( self.input.T,output_gradient)  # (output_size, input_size)
        b_gradient = np.sum(output_gradient, axis=0, keepdims=True)  # (output_size, 1).T if axis=0 else axis=1 output_size, 1
        input_gradient = np.dot(output_gradient,self.weights.T)# (input_size, batch_size)
        self.velocities_W= self.velocities_W * momentcoeff-learning_rate * w_gradient
        self.velocities_B= self.velocities_B * momentcoeff-learning_rate * b_gradient
        self.weights+=self.velocities_W
        self.bias+=self.velocities_B
        return input_gradient

In [669]:
class Activation(Layer):
    def __init__(self,activation,activatn_derivative):
        #activation is a variable pointing to a activation method,for ex sigmoid
        self.activation=activation
        self.activatn_derivative=activatn_derivative
        
    def forward(self,input):
        self.input=input
        return self.activation(self.input)
    
    def backward(self,output_gradient,learning_rate,momentcoeff):
        return self.activatn_derivative(self.input)

In [670]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1/(1+np.exp(-x))
        def sigmoid_derivative(x):
            s=sigmoid(x)
            return s*(1-s)
        super().__init__(sigmoid,sigmoid_derivative)
        

In [671]:
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        def tanh_derivative(x):
            t=tanh(x)
            return 1-(t**2)
        super().__init__(tanh,tanh_derivative)



In [672]:
# class Relu(Activation):
#     def __init(self):
#         def relu(x):
#             return max(0,x)
#         def relu_derivative(x):
#             relu=relu(x)
#             if(relu x)
            

In [673]:

class Softmax(Layer):
    def __init__(self,input_size,output_size):
        # self.weights=np.random.randn(input_size,output_size)*0.001
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / (input_size + output_size))
        self.bias=np.random.randn(1,output_size)
        self.velocities_W=np.zeros_like(self.weights)
        self.velocities_B=np.zeros_like(self.bias)
        
    def forward(self,input):
        self.input=input
        sumation=np.dot(self.input,self.weights)+self.bias
        self.softmax=np.exp(sumation)/np.sum(np.exp(sumation),axis=1, keepdims=True)
        return self.softmax
    
    def backward(self,output_gradient,learning_rate,momentcoeff):
        w_gradient = np.dot(self.input.T,output_gradient)/self.input.shape[0] # (output_size, input_size)
        b_gradient = np.sum(output_gradient, axis=0, keepdims=True)  # (output_size, 1)
        input_gradient = np.dot(output_gradient,self.weights.T)  # (input_size, batch_size)
        self.velocities_W= self.velocities_W * momentcoeff-learning_rate * w_gradient
        self.velocities_B= self.velocities_B * momentcoeff-learning_rate * b_gradient
        self.weights+=self.velocities_W
        self.bias+=self.velocities_B
        return input_gradient
        

In [674]:
class Loss(Layer):
    def __init__(self,loss_fn,loss_fn_Derivative):
        self.loss_fn=loss_fn
        self.loss_fn_Derivative=loss_fn_Derivative
    def forward(self,y_pred,y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return self.loss_fn(y_pred, y_true)
    def backward(self,y_pred,y_true):
        return self.loss_fn_Derivative(y_pred,y_true)

In [675]:
class BinaryCrossEntropy(Loss):
    def __init__(self):
        def bce(y_pred,y_true):
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)  # Prevent extreme values
            return -np.mean((y_true*np.log(y_pred))+((1-y_true)*np.log(1-y_pred)))
        def bce_derivative(y_pred,y_true):
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)  # Avoid division by zero
            return -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))
        super().__init__(bce,bce_derivative)


In [676]:
class SparseCategoricalCE(Loss):
    def __init__(self):
        def sce(logits,true_labels):
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True)) 
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            true_class_probs = probabilities[np.arange(len(true_labels)), true_labels]
            log_loss = -np.log(true_class_probs + 1e-15)  # Adding a small constant for numerical stability
            return np.mean(log_loss)
        def sce_derivative(logits, true_labels):
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Stability trick
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            batch_size = logits.shape[0]
            probabilities[np.arange(batch_size), true_labels] -= 1
            gradient = probabilities / batch_size
            
            return gradient
        super().__init__(sce,sce_derivative)
    

In [677]:

class CategoricalCrossEntropyLoss(Loss):
    def __init__(self):
        def cel(y_predSM,y_true):
            # direct correspondance of values, element wise multi
            return -np.mean(np.sum(y_true * np.log(y_predSM+ 1e-9), axis=1))
        def cel_derivative(y_predSM,y_true):
            return y_predSM-y_true
        # der wrt SM = der wrt logit(z)
        super().__init__(cel,cel_derivative)

In [678]:


class Model:
    def train(self,images,labels,layers,hyperpara):
        
        for key, value in hyperpara.items():
            setattr(self, key, value)
        self.loss_fn=CategoricalCrossEntropyLoss()
        self.X=images
        self.y=labels
        self.layers=layers
        self.softmax=Softmax(self.neurons[-1],self.softmax_neurons)
        
        for epoch in range(self.epochs):
            num_samples = self.X.shape[0]
            index=np.arange(num_samples)
            np.random.shuffle(index)
            self.X=self.X[index]
            self.y=self.y[index]
            epoch_loss=0
            for i in range(0, num_samples, self.batch_size):
                if(i+self.batch_size==num_samples):
                    X_batch = self.X[i:num_samples]
                    y_batch = self.y[i:num_samples]
                else:
                    X_batch = self.X[i:i + self.batch_size]
                    y_batch = self.y[i:i + self.batch_size]
                
                ActiPredicted=self.forward(X_batch)
                
                pred_y=self.softmax.forward(ActiPredicted)
                
                loss=self.loss_fn.forward(pred_y,y_batch)
                epoch_loss += loss
                loss_gradient=self.loss_fn.backward(pred_y,y_batch)
                # print(f"Epoch {epoch + 1}, Batch {i // self.batch_size + 1}, Loss: {loss:.4f}")

                # Backward pass
                loss_gradient=self.softmax.backward(loss_gradient,self.learning_rate,self.momentum_coeff)
                self.backward(loss_gradient)
            print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / (num_samples / self.batch_size):.4f}")
                
                
                    
                
    def forward(self,X):
        for layer in self.layers:
            # here layer is obj to class not class*
            X=layer.forward(X)
        return X
        

    def backward(self, loss_gradient):
        for layer in reversed(self.layers):
            loss_gradient = layer.backward(loss_gradient, self.learning_rate,self.momentum_coeff)
    def predict(self, X):
        logits = self.forward(X)
        probabilities = self.softmax.forward(logits)
        return np.argmax(probabilities, axis=1)

    def evaluate(self, X, y):
        predictions = self.predict(X)
        true_labels = np.argmax(y, axis=1)
        accuracy = np.mean(predictions == true_labels)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        return accuracy

        

In [679]:
hyperparameters={
    "hiddenLayers":2,
    "learning_rate":0.0001,
    "batch_size":25,
    "epochs":16,
    "ActivationFns":None,
    "neurons":None,
    "softmax_neurons":labels.shape[1],
    "momentum_coeff":0.9
    # v=coeff*v-learnrate*gradient
    # w=w+v
    
}
# change
hyperparameters["neurons"]=[32,16]
hyperparameters["ActivationFns"]=[Sigmoid(),Tanh()]
layers=[]
inputsize=images.shape[1]
for i in range(hyperparameters["hiddenLayers"]):
    outputsize=hyperparameters["neurons"][i]
    layers.extend([Dense(inputsize,outputsize),hyperparameters["ActivationFns"][i]])
    inputsize=outputsize


model=Model()
model.train(images,labels,layers,hyperparameters)

Epoch 1, Average Loss: 2.5340
Epoch 2, Average Loss: 2.4297
Epoch 3, Average Loss: 2.3566
Epoch 4, Average Loss: 2.3010
Epoch 5, Average Loss: 2.2528
Epoch 6, Average Loss: 2.2123
Epoch 7, Average Loss: 2.1801
Epoch 8, Average Loss: 2.1540
Epoch 9, Average Loss: 2.1344
Epoch 10, Average Loss: 2.1184
Epoch 11, Average Loss: 2.1065
Epoch 12, Average Loss: 2.0964
Epoch 13, Average Loss: 2.0888
Epoch 14, Average Loss: 2.0829
Epoch 15, Average Loss: 2.0783
Epoch 16, Average Loss: 2.0749


In [680]:
print(f"accuary:{model.evaluate(test_images,test_labels)}")

Accuracy: 14.16%
accuary:0.14163090128755365
